In [1]:
from datetime import date
from pprint import pprint
from xml.dom.minidom import parseString as xml_parse

from rest_framework.renderers import JSONRenderer
from rest_framework_xml.renderers import XMLRenderer

from organizer.models import NewsLink, Startup, Tag
from organizer.serializers import NewsLinkSerializer, StartupSerializer, TagSerializer

# Simple example with Tag object

In [38]:
new_tag = Tag.objects.create(name='dango')

IntegrityError: duplicate key value violates unique constraint "organizer_tag_name_key"
DETAIL:  Key (name)=(dango) already exists.


In [39]:
s_tag = TagSerializer(new_tag)

In [40]:
s_tag

TagSerializer(<Tag: dango>):
    id = IntegerField(read_only=True)
    name = CharField(max_length=31)
    slug = SlugField(allow_blank=True, max_length=31)

In [41]:
#outputs a python dictionary
s_tag.data

{'id': 1, 'name': 'dango', 'slug': 'dango'}

In [7]:
type(s_tag.data)

rest_framework.utils.serializer_helpers.ReturnDict

In [9]:
#This is a json type
json = JSONRenderer().render(s_tag.data)
#it is being printed as bytes
json

b'{"id":1,"name":"dango","slug":"dango"}'

In [16]:
#Helper
def render_json(serialized_object):
    print(
        JSONRenderer().render(
            serialized_object.data,
            accepted_media_type='application/json;indent=4'
        ).decode('utf8')
    )

In [17]:
render_json(s_tag)

{
    "id": 1,
    "name": "dango",
    "slug": "dango"
}


In [19]:
#Example of rendering in XML
XMLRenderer().render(s_tag.data)

'<?xml version="1.0" encoding="utf-8"?>\n<root><id>1</id><name>dango</name><slug>dango</slug></root>'

In [20]:
def render_xml(serialized_object):
    print(
        xml_parse(
            XMLRenderer().render(
                serialized_object.data
            )
        ).toprettyxml()
    )

In [21]:
render_xml(s_tag)

<?xml version="1.0" ?>
<root>
	<id>1</id>
	<name>dango</name>
	<slug>dango</slug>
</root>



# Serialize object with Relationships

In [25]:
#Creating a startup object
jambon = Startup.objects.create(
    name='Jam Software',
    slug='jam-software',
    description= 'Software Consulting & Training',
    founded_date= date(2013,1,18),
    contact='django@jamsw.com',
    website='https://www.jamsoft.com',
)

IntegrityError: duplicate key value violates unique constraint "organizer_startup_slug_key"
DETAIL:  Key (slug)=(jam-software) already exists.


In [26]:
#Add tags: django, web
jambon.tags.add(new_tag, Tag.objects.create(name='web'))

In [28]:
#Serialize the data
s_jambon= StartupSerializer(jambon)

In [29]:
s_jambon

StartupSerializer(<Startup: Jam Software>):
    id = IntegerField(read_only=True)
    name = CharField(max_length=31)
    slug = SlugField(max_length=31)
    description = CharField()
    founded_date = DateField()
    contact = EmailField()
    website = URLField(max_length=255)
    tags = TagSerializer(many=True):
        id = IntegerField(read_only=True)
        name = CharField(max_length=31)
        slug = SlugField(allow_blank=True, max_length=31)

In [30]:
pprint(s_jambon.data)

{'contact': 'django@jamsw.com',
 'description': 'Software Consulting & Training',
 'founded_date': '2013-01-18',
 'id': 1,
 'name': 'Jam Software',
 'slug': 'jam-software',
 'tags': [OrderedDict([('id', 1), ('name', 'dango'), ('slug', 'dango')]),
          OrderedDict([('id', 2), ('name', 'web'), ('slug', 'web')])],
 'website': 'https://www.jamsoft.com'}


In [31]:
render_json(s_jambon)

{
    "id": 1,
    "name": "Jam Software",
    "slug": "jam-software",
    "description": "Software Consulting & Training",
    "founded_date": "2013-01-18",
    "contact": "django@jamsw.com",
    "website": "https://www.jamsoft.com",
    "tags": [
        {
            "id": 1,
            "name": "dango",
            "slug": "dango"
        },
        {
            "id": 2,
            "name": "web",
            "slug": "web"
        }
    ]
}


In [32]:
render_xml(s_jambon)

<?xml version="1.0" ?>
<root>
	<id>1</id>
	<name>Jam Software</name>
	<slug>jam-software</slug>
	<description>Software Consulting &amp; Training</description>
	<founded_date>2013-01-18</founded_date>
	<contact>django@jamsw.com</contact>
	<website>https://www.jamsoft.com</website>
	<tags>
		<list-item>
			<id>1</id>
			<name>dango</name>
			<slug>dango</slug>
		</list-item>
		<list-item>
			<id>2</id>
			<name>web</name>
			<slug>web</slug>
		</list-item>
	</tags>
</root>



In [33]:
#Now try with a newslink
nl= NewsLink(
    title='JamBon Software rated best ever',
    slug='jambon-best',
    pub_date=date(2018,4,1),
    link='https://www.xkcd.com/252',
    startup=jambon
)

In [34]:
s_nl = NewsLinkSerializer(nl)
s_nl

NewsLinkSerializer(<NewsLink: Jam Software: JamBon Software rated best ever>):
    id = IntegerField(read_only=True)
    title = CharField(max_length=63)
    slug = SlugField(max_length=63)
    pub_date = DateField()
    link = URLField(max_length=255)
    startup = StartupSerializer():
        id = IntegerField(read_only=True)
        name = CharField(max_length=31)
        slug = SlugField(max_length=31)
        description = CharField()
        founded_date = DateField()
        contact = EmailField()
        website = URLField(max_length=255)
        tags = TagSerializer(many=True):
            id = IntegerField(read_only=True)
            name = CharField(max_length=31)
            slug = SlugField(allow_blank=True, max_length=31)

In [35]:
#Print as dictionary
pprint(s_nl.data)

{'id': None,
 'link': 'https://www.xkcd.com/252',
 'pub_date': '2018-04-01',
 'slug': 'jambon-best',
 'startup': OrderedDict([('id', 1),
                         ('name', 'Jam Software'),
                         ('slug', 'jam-software'),
                         ('description', 'Software Consulting & Training'),
                         ('founded_date', '2013-01-18'),
                         ('contact', 'django@jamsw.com'),
                         ('website', 'https://www.jamsoft.com'),
                         ('tags',
                          [OrderedDict([('id', 1),
                                        ('name', 'dango'),
                                        ('slug', 'dango')]),
                           OrderedDict([('id', 2),
                                        ('name', 'web'),
                                        ('slug', 'web')])])]),
 'title': 'JamBon Software rated best ever'}


In [36]:
render_json(s_nl)

{
    "id": null,
    "title": "JamBon Software rated best ever",
    "slug": "jambon-best",
    "pub_date": "2018-04-01",
    "link": "https://www.xkcd.com/252",
    "startup": {
        "id": 1,
        "name": "Jam Software",
        "slug": "jam-software",
        "description": "Software Consulting & Training",
        "founded_date": "2013-01-18",
        "contact": "django@jamsw.com",
        "website": "https://www.jamsoft.com",
        "tags": [
            {
                "id": 1,
                "name": "dango",
                "slug": "dango"
            },
            {
                "id": 2,
                "name": "web",
                "slug": "web"
            }
        ]
    }
}


In [37]:
render_xml(s_nl)

<?xml version="1.0" ?>
<root>
	<id/>
	<title>JamBon Software rated best ever</title>
	<slug>jambon-best</slug>
	<pub_date>2018-04-01</pub_date>
	<link>https://www.xkcd.com/252</link>
	<startup>
		<id>1</id>
		<name>Jam Software</name>
		<slug>jam-software</slug>
		<description>Software Consulting &amp; Training</description>
		<founded_date>2013-01-18</founded_date>
		<contact>django@jamsw.com</contact>
		<website>https://www.jamsoft.com</website>
		<tags>
			<list-item>
				<id>1</id>
				<name>dango</name>
				<slug>dango</slug>
			</list-item>
			<list-item>
				<id>2</id>
				<name>web</name>
				<slug>web</slug>
			</list-item>
		</tags>
	</startup>
</root>

